In [1]:
import os

Reading API Keys

In [2]:
#Reading OpenAI Key
OPENAI_API_KEY=input("Enter OpenAI API Key")
#Reading Langchain API Key
LANGCHAIN_API_KEY=input("Enter Langchain API Key")

In [3]:
#Enabiling Langchain Tracing
LANGCHAIN_TRACING_V2= "true"
#OpenAI Model
OPENAI_MODEL="gpt-4o-mini"

In [ ]:
# installing langchain_openai package
! pip install langchain_openai

In [5]:
# initializing llm
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model=OPENAI_MODEL,openai_api_key=OPENAI_API_KEY,temperature=0.5)
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10d145430>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10d146930>, root_client=<openai.OpenAI object at 0x10cc658e0>, root_async_client=<openai.AsyncOpenAI object at 0x10d145460>, model_name='gpt-4o-mini', temperature=0.5, openai_api_key=SecretStr('**********'), openai_proxy='')

In [ ]:
# Installing Python package for SQLDatabase
! pip install langchain-community==0.2.9
! pip install sqlalchemy==2.0.28

In [8]:
#initializing Database object
from langchain_community.utilities.sql_database import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///baggage_tracking.db")
db